In [1]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets
!pip install flash-attn --no-build-isolation

     ---------------------------------------- 0.0/138.0 kB ? eta -:--:--
     -- ------------------------------------- 10.2/138.0 kB ? eta -:--:--
     -------- ---------------------------- 30.7/138.0 kB 435.7 kB/s eta 0:00:01
     ------------------------------------ - 133.1/138.0 kB 1.3 MB/s eta 0:00:01
     ------------------------------------ - 133.1/138.0 kB 1.3 MB/s eta 0:00:01
     ------------------------------------ - 133.1/138.0 kB 1.3 MB/s eta 0:00:01
     ------------------------------------ 138.0/138.0 kB 511.0 kB/s eta 0:00:00
  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
  Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   -- ------------------------------------- 0.7/9.0 MB 13.7 MB/s eta 0:00:01
   ------ ---------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faster-whisper 1.0.1 requires tokenizers<0.16,>=0.13, but you have tokenizers 0.19.1 which is incompatible.


     ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
     ---------- ----------------------------- 0.6/2.5 MB 20.1 MB/s eta 0:00:01
     ----------------------------- ---------- 1.8/2.5 MB 23.5 MB/s eta 0:00:01
     ------------------------------ --------- 1.9/2.5 MB 14.9 MB/s eta 0:00:01
     ------------------------------- -------- 1.9/2.5 MB 11.3 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 13.1 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 13.1 MB/s eta 0:00:01
     ---------------------------------------- 2.5/2.5 MB 8.8 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [17 lines of output]
  Traceback (most recent call last):
    File "C:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 149, in prepare_metadata_for_build_wheel
      return hook(metadata_directory, config_settings)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\sit

In [2]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("mosaicml/instruct-v3")

Generating train split:   0%|          | 0/56167 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [3]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 56167
    })
    test: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 6807
    })
})

In [4]:
instruct_tune_dataset = instruct_tune_dataset.filter(lambda x: x["source"] == "dolly_hhrlhf")

Filter:   0%|          | 0/56167 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [5]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 34333
    })
    test: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 4771
    })
})

In [6]:
instruct_tune_dataset["train"][0]

{'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction\nWhat are different types of grass?\n\n### Response\n',
 'response': 'There are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.',
 'source': 'dolly_hhrlhf'}

In [7]:
def create_prompt(sample):
    bos_token = "<s>"
    original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    system_message = "[INST Use the provided input to create an instruction that could have been used to generate the response with the LLM."
    response = sample["prompt"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
    input = sample["response"]
    eos_token = "</s>"
    
    full_prompt = ""
    full_prompt += bos_token
    full_prompt += system_message
    full_prompt += "\n" + input
    full_prompt += "[/INST]"
    full_prompt += response
    full_prompt += eos_token
    
    return full_prompt    

In [8]:
create_prompt(instruct_tune_dataset["train"][0])

'<s>[INST Use the provided input to create an instruction that could have been used to generate the response with the LLM.\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.[/INST]What are different types of grass?</s>'

Loading the Base Model

In [1]:
model_id = "mistralai/Mixtral-8x7B-v0.1"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [15]:
!huggingface-cli login

^C


In [6]:
!pip install accelerate

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=nf4_config,
    use_cache=False,
    attn_implementation="flash_attention_2"
)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to("cuda")

In [4]:
from huggingface_hub import notebook_login

notebook_login()